In [25]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# read csv file
local_guide = pd.read_csv('local_guide.csv')
local_guide.head()

,Pemandu_ID,Nama_Pemandu,Optional_Bahasa,Umur,Jenis_Kelamin,Tempat,Pendidikan_Terakhir,Pekerjaan,Nomor_Telepon,Price_per_hour,Time_duration_in_min,Rating,Avatars
0,PMD000,Kani Puspasari,Inggris,23,Perempuan,South Solok,SD,Wartawan,6283760988551,50000,300,3,xsgames.co/randomusers/avatar.php?g=female
1,PMD001,Lega Hardiansyah,Inggris,18,Laki-laki,Gowa,SMA,Wiraswasta,6284225892082,66000,360,3,xsgames.co/randomusers/avatar.php?g=male
2,PMD002,"Ir. Zulfa Iswahyudi, S.E.I",Jepang,32,Perempuan,Kabupaten Gunung Kidul,S1,Guru,6286605735920,82000,60,4,xsgames.co/randomusers/avatar.php?g=female
3,PMD003,Icha Kusumo,Mandarin,38,Laki-laki,Bogor,S1,Pengusaha,6281121979913,126000,240,2,xsgames.co/randomusers/avatar.php?g=male
4,PMD004,KH. Ridwan Hariyah,Inggris,26,Laki-laki,Moskva,S1,Guru,6282581398149,143000,180,4,xsgames.co/randomusers/avatar.php?g=male


In [27]:
# info of local_guide
local_guide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Pemandu_ID            2000 non-null   object
 1   Nama_Pemandu          2000 non-null   object
 2   Optional_Bahasa       2000 non-null   object
 3   Umur                  2000 non-null   int64 
 4   Jenis_Kelamin         2000 non-null   object
 5   Tempat                2000 non-null   object
 6   Pendidikan_Terakhir   2000 non-null   object
 7   Pekerjaan             2000 non-null   object
 8   Nomor_Telepon         2000 non-null   int64 
 9   Price_per_hour        2000 non-null   int64 
 10  Time_duration_in_min  2000 non-null   int64 
 11  Rating                2000 non-null   int64 
 12  Avatars               2000 non-null   object
dtypes: int64(5), object(8)
memory usage: 203.2+ KB


In [28]:
# data
data = pd.read_csv('local_guide.csv')

# create object TfidfVectorizer
vectorizer = TfidfVectorizer()

# Melakukan vektorisasi TF-IDF pada fitur "Tempat"
tfidf_matrix_tempat = vectorizer.fit_transform(data['Tempat'])

# Mengubah matriks TF-IDF menjadi array
tfidf_matrix = tfidf_matrix_tempat.toarray()

# Mendapatkan indeks item berdasarkan input Tempat
def get_item_index_by_tempat(tempat, data):
    index = data[data['Tempat'] == tempat].index
    return index[0] if len(index) > 0 else None

# Membangun model sequential
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(tfidf_matrix.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(tfidf_matrix.shape[1], activation='linear')
])

# summary model
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               20736     
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 161)               5313      
                                                                 
Total params: 36,385
Trainable params: 36,385
Non-trainable params: 0
_________________________________________________________________


In [29]:
# compile model
model.compile(loss='cosine_similarity', optimizer='adam')

In [30]:
# fit model
model.fit(tfidf_matrix, tfidf_matrix, epochs=10, batch_size=32)

Epoch 1/10
63/63 [==============================] - 4s 7ms/step - loss: -0.2234
Epoch 2/10
63/63 [==============================] - 0s 6ms/step - loss: -0.4116
Epoch 3/10
63/63 [==============================] - 0s 7ms/step - loss: -0.5003
Epoch 4/10
63/63 [==============================] - 1s 9ms/step - loss: -0.5437
Epoch 5/10
63/63 [==============================] - 1s 8ms/step - loss: -0.5672
Epoch 6/10
63/63 [==============================] - 0s 6ms/step - loss: -0.5813
Epoch 7/10
63/63 [==============================] - 0s 6ms/step - loss: -0.5917
Epoch 8/10
63/63 [==============================] - 0s 6ms/step - loss: -0.5981
Epoch 9/10
63/63 [==============================] - 0s 6ms/step - loss: -0.6039
Epoch 10/10
63/63 [==============================] - 0s 6ms/step - loss: -0.6081


In [31]:
# savings models
model.save('model_local_guide.h5')